# HySPI
## Industrial Hydrogen - Prospective Scenarios for Environmental impacts
### Notebook developed by Juliana Steinbach, Joanna Schlesinger and Thomas Beaussier
#### Version 1.0 - November 2023

In [1]:
import bw2analyzer as bwa
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import SALib
from tabulate import tabulate
from sympy import *
from scipy.stats import binned_statistic
import seaborn as sns
from IPython.display import HTML, display
import bw2io
import brightway2 as bw
import lca_algebraic as agb
from lca_algebraic import*
from brightway2 import projects, methods, databases, methods

# from lca_algebraic.params import FixedParamMode
# from lca_algebraic.stats import *
# from lca_algebraic.stats import _generate_random_params, _compute_stochastics

from IPython.core.display import display, HTML

# Larger space in notebook for large graphs
#display(HTML("<style>.container { width:70% !important; }</style>"))

# Some options for pandas formatting
pd.options.display.max_rows = 100
pd.options.display.float_format = '{:,.3g}'.format

## Setup new project

In [2]:
#agb.initProject('HySIP_2050')
agb.initProject('RTE_M0')
#RTE_test5

## Importing databases

In [3]:
# Import Ecoinvent DB
#agb.importDb("ei_3.9", r'C:\Users\jsteinbach\Desktop\juliana\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets')

In [4]:
# Creation of the local DB with resting of all parameters
USER_DB='user-db'
PREM_DBPM0='ecoinvent_cutoff_3.9_image_SSP2-Base_2050_M0'
PREM_DBPM1='ecoinvent_cutoff_3.9_image_SSP2-Base_2050_M1'
PREM_DBPM23='ecoinvent_cutoff_3.9_image_SSP2-Base_2050_M23'
PREM_DBPN1='ecoinvent_cutoff_3.9_image_SSP2-Base_2050_N1'
PREM_DBPN2='ecoinvent_cutoff_3.9_image_SSP2-Base_2050_N2'
PREM_DBPN03='ecoinvent_cutoff_3.9_image_SSP2-Base_2050_N03'
EI="ei_3.9"
agb.resetDb(USER_DB)
agb.resetParams()


In [5]:
#import PREMISE database from .xlsx
imp = bw.ExcelImporter(r"HySPI_LCI_3.9.xlsx") 
del imp.strategies[6]
del imp.strategies[6]

imp.apply_strategies()
imp.match_database(EI, fields=('name', 'reference product', 'unit', 'location')) 
imp.match_database("biosphere3", fields=('name', 'unit', 'categories')) 
imp.match_database(fields=['name'])

#to verify what was don
imp.statistics()
#if unlinked flows exist
imp.write_excel(only_unlinked=True)

imp.write_database()

In [6]:
bw.databases

In [7]:
#Delete DB if needed
#bw.databases["h2_pem"]

## Choice of the method

In [8]:
# Choice of the method
EF = 'EF v3.0 no LT'

#Search impact methods : agb.findMethods("climate", EF)
#See all impact methods and impacts of brightway2 :
#list(bw.methods)
agb.findMethods("eutrophication", EF)

In [9]:
#Choices of the impacts
climate = (EF, 'climate change no LT', 'global warming potential (GWP100) no LT')
resources = (EF, 'material resources: metals/minerals no LT', 'abiotic depletion potential (ADP): elements (ultimate reserves) no LT')
sol = (EF, 'land use no LT', 'soil quality index no LT')
water = (EF, 'water use no LT', 'user deprivation potential (deprivation-weighted water consumption) no LT')
acidification = (EF, 'acidification no LT', 'accumulated exceedance (AE) no LT')
marine_eutroph = (EF, 'eutrophication: marine no LT', 'fraction of nutrients reaching marine end compartment (N) no LT')
freshwater_eutroph = (EF, 'eutrophication: freshwater no LT', 'fraction of nutrients reaching freshwater end compartment (P) no LT')
terre_eutroph = (EF, 'eutrophication: terrestrial no LT', 'accumulated exceedance (AE) no LT')
radiation = (EF, 'ionising radiation: human health no LT', 'human exposure efficiency relative to u235 no LT')

#Test if the chosen impacts are impact cathegories
#bw.methods[radiation]

# List of the impacts
impacts = [climate, resources, sol, water, acidification, marine_eutroph, freshwater_eutroph, terre_eutroph, radiation]
#impacts

## Building the inventory by adapting ecoinvent 3.9
### List of the inputs to be adapted

In [10]:
Elec_FR_2023= agb.findActivity("market for electricity, low voltage", loc= "FR", single= False, db_name="ei_3.9") #base for comparison

In [11]:
nuclear= agb.findActivity("electricity production, nuclear, pressure water reactor*", loc='FR', db_name=EI)

hydro_alpine= agb.findActivity("electricity production, hydro, reservoir, alpine region", loc='FR', db_name=EI)
hydro_river= agb.findActivity("electricity production, hydro, run-of-river", loc='FR', db_name=EI)

solar_multi= agb.findActivity("electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted", loc='FR', single=False, db_name=EI)
solar_single= agb.findActivity("electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted", loc='FR', single=False, db_name=EI)
solar_ground= agb.findActivity("electricity production, photovoltaic, 570kWp open ground installation, multi-Si", loc='FR', single=False, db_name=EI)

wind_on_1_3= agb.findActivity("electricity production, wind, 1-3MW turbine, onshore", loc='FR', db_name=EI)
wind_on_1= agb.findActivity("electricity production, wind, <1MW turbine, onshore", loc='FR', db_name=EI)
wind_on_3= agb.findActivity("electricity production, wind, >3MW turbine, onshore", loc='FR', db_name=EI)
wind_off_1_3_fixed= agb.findActivity("electricity production, wind, 1-3MW turbine, offshore", loc='FR', db_name=EI)
wind_off_1_3_float= agb.findActivity("electricity production, wind, 1-3MW turbine, offshore", loc='FR', db_name=EI) #until we have the foating one it will be used the fixed as proxy
#data provided by Tsai et al. 2016 for fixed foundations of 3 MW wind turbines and Weinzettel et al., 2008 for a spar floating foundations of 5 MW wind turbines.

bioenergy= agb.findActivity("heat and power co-generation, wood chips, 6667 kW", unit= 'kilowatt hour', loc= 'CH', db_name=EI)
cable_aero= agb.findActivity("market for transmission network, electricity, high voltage direct current aerial line", loc='GLO',  db_name=EI)
cable_land= agb.findActivity("market for transmission network, electricity, high voltage direct current land cable", loc='GLO', db_name=EI)
cable_subsea= agb.findActivity("distribution network construction, electricity, low voltage", loc='CH', db_name=EI)

rest=agb.findActivity("market for electricity, high voltage", loc='FR',  db_name=EI)
#rest=agb.findActivity("market for electricity, high voltage", loc='IS',  db_name=EI) #In case we want to use an inventory that does not contain any nuclear source and it is highly decarbonized

sulfur=agb.findActivity("market for sulfur hexafluoride, liquid", loc='RER', db_name=EI)

transf_M=agb.findActivity("market for transmission network, electricity, medium voltage", loc='GLO', db_name=EI)
dist_L=agb.findActivity("market for distribution network, electricity, low voltage", loc='GLO', db_name=EI)


water_H2=agb.findActivity("market for water, deionised", loc='Europe without Switzerland',  db_name=EI)

#battery=agb.findActivity("electricity production, from stationary battery, FE2050", loc='FR', db_name=PREM_DBPM0)
#hydrogen= battery=agb.findActivity("electricity production, from hydrogen, FE2050", loc='FR', db_name=PREM_DBPM0)

In [12]:
#create electricity inventory for FR 2050 based on M0 data and ecoinvent
Elec_FR_2050_DM0=agb.newActivity(USER_DB,
                        name="Electricity mix, FR, 2050, RTE Scenario M0, low voltage",
                        unit = "kilowatt hour",
                        loc='FR',
                        )

In [13]:
#create electricity inventory for FR 2050 based on M1 data and ecoinvent
Elec_FR_2050_DM1=agb.newActivity(USER_DB,
                        name="Electricity mix, FR, 2050, RTE Scenario M1, low voltage",
                        unit = "kilowatt hour",
                        loc='FR',
                        )

In [14]:
#create electricity inventory for FR 2050 based on M23 data and ecoinvent
Elec_FR_2050_DM23=agb.newActivity(USER_DB,
                        name="Electricity mix, FR, 2050, RTE Scenario M23, low voltage",
                        unit = "kilowatt hour",
                        loc='FR',
                        )

In [15]:
#transformation factors from high to medium to low voltage
PGL_HM=1.069518717 #grid loss from high to medium in premise
PGL_ML=1.027749229 #grid loss from medium to low in premise
EIGL_HM=1.06 #grid loss from high to medium in ecoinvent
EIGL_ML=1.03 #grid loss from medium to low in ecoinvent

In [16]:
Elec_FR_2050_DM0.addExchanges({
    nuclear:0,
    hydro_alpine:0.016510667*EIGL_HM*EIGL_ML,
    hydro_river:0.086683436*EIGL_HM*EIGL_ML,
    solar_multi:0.073958243,
    solar_single:0.05946433,
    solar_ground:0.19568644,
    wind_on_1_3:0.18070826*EIGL_HM*EIGL_ML,
    wind_on_1:0.011613871*EIGL_HM*EIGL_ML,
    wind_on_3: 0.000746527*EIGL_HM*EIGL_ML,
    wind_off_1_3_fixed:0.115608431*EIGL_HM*EIGL_ML,
    wind_off_1_3_float:0.173412647*EIGL_HM*EIGL_ML,
    bioenergy:0.01267296*EIGL_HM*EIGL_ML,
    cable_aero:5.65463552361011E-09,
    cable_land:3.86E-10,
    cable_subsea:3.47E-12,
    #battery:0.035561878*EIGL_HM*EIGL_ML,  #in case we decide to use the inventory from premise
    #hydrogen:0.020690547*EIGL_HM*EIGL_ML,  #in case we decide to use the inventory from premise
    #rest:0.016681754*EIGL_HM*EIGL_ML/0.5,  #in case we decide to use the inventory from premise
    rest:0.072934178*EIGL_HM*EIGL_ML/0.5,
    sulfur:5.4e-08*EIGL_ML+4.03e-9, #sulfur content in premise is e-09 and in ecoinvent is e-10
    transf_M:1.86277676887616e-08, #same as ecoinvent
    dist_L:8.74048809653223e-08,  #same as ecoinvent
    water_H2: 0.020690547*14/0.25, #14L of water, fuell cell efficiency (50% 0,5kW), eliminate this one in case you use hydrogen data from premise
})

In [17]:
Elec_FR_2050_DM1.addExchanges({
    nuclear:0.120084735*EIGL_HM*EIGL_ML,
    hydro_alpine:0.016840593*EIGL_HM*EIGL_ML,
    hydro_river:0.088415596*EIGL_HM*EIGL_ML,
    solar_multi:0.093535749,
    solar_single:0.075205148,
    solar_ground:0.168740898,
    wind_on_1_3:0.147094848*EIGL_HM*EIGL_ML,
    wind_on_1:0.009453583*EIGL_HM*EIGL_ML,
    wind_on_3: 0.000607666*EIGL_HM*EIGL_ML,
    wind_off_1_3_fixed:0.086920513*EIGL_HM*EIGL_ML,
    wind_off_1_3_float:0.128093388*EIGL_HM*EIGL_ML,
    bioenergy:0.012974977*EIGL_HM*EIGL_ML,
    cable_aero:5.65463552361011E-09,
    cable_land:3.86E-10,
    cable_subsea:3.47E-12,
    #battery:0.028730306*EIGL_HM*EIGL_ML,
    #hydrogen:0.013901761*EIGL_HM*EIGL_ML,
    #rest:0.009400238*EIGL_HM*EIGL_ML/0.5,
    rest:0.052032305*EIGL_HM*EIGL_ML/0.5,
    sulfur:5.4e-08*EIGL_ML+4.03e-9, #sulfur content in premise is e-09 and in ecoinvent is e-10
    transf_M:1.86277676887616e-08,
    dist_L:8.74048809653223e-08,
    water_H2: 0.013901761*14/0.24, #14L of water, fuell cell efficiency 50%
})

In [18]:
Elec_FR_2050_DM23.addExchanges({
    nuclear:0.125430263*EIGL_HM*EIGL_ML,
    hydro_alpine:0.017226635*EIGL_HM*EIGL_ML,
    hydro_river:0.090442372*EIGL_HM*EIGL_ML,
    solar_multi:0.041461943,
    solar_single:0.033336469,
    solar_ground:0.135858341,
    wind_on_1_3:0.187247401*EIGL_HM*EIGL_ML,
    wind_on_1:0.012034132*EIGL_HM*EIGL_ML,
    wind_on_3: 0.000773541*EIGL_HM*EIGL_ML,
    wind_off_1_3_fixed:0.117632532*EIGL_HM*EIGL_ML,
    wind_off_1_3_float:0.178801449*EIGL_HM*EIGL_ML,
    bioenergy:0.013493047*EIGL_HM*EIGL_ML,
    cable_aero:5.65463552361011E-09,
    cable_land:3.86E-10,
    cable_subsea:3.47E-12,
    #battery:0.01624673*EIGL_HM*EIGL_ML,
    #hydrogen:0.01225389*EIGL_HM*EIGL_ML,
    #rest:0.017761256*EIGL_HM*EIGL_ML/0.5,
    rest:0.046261875*EIGL_HM*EIGL_ML/0.5,
    sulfur:5.4e-08*EIGL_ML+4.03e-9, #sulfur content in premise is e-09 and in ecoinvent is e-10
    transf_M:1.86277676887616e-08,
    dist_L:8.74048809653223e-08,
    water_H2: 0.01225389*14/0.24, #14L of water, fuell cell efficiency 50%
})

In [19]:
Elec_FR_2050_PM0= agb.findActivity("market for electricity, low voltage, FE2050*", single= False, db_name=PREM_DBPM0)

In [20]:
Elec_FR_2050_PM1= agb.findActivity("market for electricity, low voltage, FE2050*", single= False, db_name=PREM_DBPM1)

In [21]:
Elec_FR_2050_PM23= agb.findActivity("market for electricity, low voltage, FE2050*", single= False, db_name=PREM_DBPM23)

In [22]:
Elec_FR_2050_PN1= agb.findActivity("market for electricity, low voltage, FE2050*", single= False, db_name=PREM_DBPN1)

In [23]:
Elec_FR_2050_PN2= agb.findActivity("market for electricity, low voltage, FE2050*", single= False, db_name=PREM_DBPN2)

In [24]:
Elec_FR_2050_PN03= agb.findActivity("market for electricity, low voltage, FE2050*", single= False, db_name=PREM_DBPN03)

In [25]:
Elec_solar_DM0=agb.newActivity(USER_DB,
                        name="Electricity mix, FR, 2050, Solar ground pannel dedicated system + RTE Scenario M0, low voltage",
                        unit = "kilowatt hour",
                        loc='FR',
                        )

In [26]:
Elec_solar_PM0=agb.newActivity(USER_DB,
                        name="Electricity mix, FR, 2050, Solar ground pannel dedicated system + RTE Scenario PM0, low voltage",
                        unit = "kilowatt hour",
                        loc='FR',
                        )

In [27]:
Elec_solar_DM1=agb.newActivity(USER_DB,
                        name="Electricity mix, FR, 2050, Solar ground pannel dedicated system + RTE Scenario M1, low voltage",
                        unit = "kilowatt hour",
                        loc='FR',
                        )

In [28]:
Elec_solar_PM1=agb.newActivity(USER_DB,
                        name="Electricity mix, FR, 2050, Solar ground pannel dedicated system + RTE Scenario PM1, low voltage",
                        unit = "kilowatt hour",
                        loc='FR',
                        )

In [29]:
Elec_solar_DM23=agb.newActivity(USER_DB,
                        name="Electricity mix, FR, 2050, Solar ground pannel dedicated system + RTE Scenario M23, low voltage",
                        unit = "kilowatt hour",
                        loc='FR',
                        )

In [30]:
Elec_solar_PM23=agb.newActivity(USER_DB,
                        name="Electricity mix, FR, 2050, Solara ground pannel dedicated system + RTE Scenario PM23, low voltage",
                        unit = "kilowatt hour",
                        loc='FR',
                        )

In [31]:
Elec_solar_DM0.addExchanges({
    solar_ground:0.17,
    Elec_FR_2050_DM0: 0.83
})

In [32]:
Elec_solar_PM0.addExchanges({
    solar_ground:0.17,
    Elec_FR_2050_PM0: 0.83
})

In [33]:
Elec_solar_DM1.addExchanges({
    solar_ground:0.17,
    Elec_FR_2050_DM1: 0.83
})

In [34]:
Elec_solar_PM1.addExchanges({
    solar_ground:0.17,
    Elec_FR_2050_PM1: 0.83
})

In [35]:
Elec_solar_DM23.addExchanges({
    solar_ground:0.17,
    Elec_FR_2050_DM23: 0.83
})

In [36]:
Elec_solar_PM23.addExchanges({
    solar_ground:0.17,
    Elec_FR_2050_PM23: 0.83
})

## Electricity inventory visualization

In [37]:
agb.printAct(Elec_FR_2050_DM0)

In [38]:
agb.printAct(Elec_FR_2050_DM1)

In [39]:
agb.printAct(Elec_FR_2050_DM23)

## electrolyzer inventory

In [40]:
#electrolyzer
PEM_Stack=agb.findActivity(name='electrolyzer production, 1MWe, PEM, Stack', db_name='AEC/PEM')
PEM_BoP=agb.findActivity(name='electrolyzer production, 1MWe, PEM, Balance of Plant', db_name='AEC/PEM')

AEC_Stack=agb.findActivity(name='electrolyzer production, 1MWe, AEC, Stack', db_name='AEC/PEM')
AEC_BoP=agb.findActivity(name='electrolyzer production, 1MWe, AEC, Balance of Plant', db_name='AEC/PEM')

### Data source for parametrization

In [41]:
#current_life_stack_PEM/AEC= avarage from Future cost and performance of water electrolysis: An expert elicitation study
#future_life_stack_PEM/AEC= retrieved from Life cycle assessment of hydrogen from proton exchange membrane water electrolysis in future energy systems
#current/future_eff_PEM/AEC=retrieved from graph apendix E Future cost and performance of water electrolysis

#Hydrogen data
#Z=1.126 #compressibility factor
#molar_mass_H2=0.00201588 #kg/mol
#gamma=1.41
#R=8.314 #J/mol/K
#H2_density = 0.08988 #kg/m^3 at standard condition
#LHV = 286 kJ/mol or 120 MJ/kg
#100% efficiency 1kWh = 0.03768 kg H2

#Electrolysis
water_need=14  #kg or L because it is deionised water with 1kg/L
PCI_H2=33.33 #kWh/kgH2

H2_produced_20_AEC=4114614.82 #kg # Gerloff uses 3085961kg produced in 15 years, here we linearly adapted it to 20y, premise table uses 4259200kg
H2_produced_20_PEM=2964315.35 #kg # Gerloff, 20y, no adaptation, premise table uses 29632000kg
# RH= H2_produced_20_AEC/H2_produced_20_PEM= 1.388
# Guo 2019 states 1000kg/h for AEC and 400 for PEM, RH= 2.5
#36500000 kgH2/year Palmer > in 20 years = 730 000 000

#Electrolyser PEM
current_life_stack_PEM= 60000 #h
future_life_stack_PEM= 90000 #h
current_eff_PEM=0.61
future_eff_PEM=0.79
water_demand_PEM=14

#Electrolyser AEC
current_life_stack_AEC= 75000 #h
future_life_stack_AEC= 90000 #h
current_eff_AEC=0.64
future_eff_AEC=0.79
water_demand_AEC=14
# Palmer used 55kWh/H2

#output pressure post production: 80 bar Palmer 2021

## Parametrization

In [42]:
p_electricity=agb.newEnumParam( 
    "p_electricity",                     # Short name
    label="electricity mix",        # English label
    description="M0, M1 and M23 RTE scenarios for FR low voltage", # Long description  
    values = ["E2023","DM0","DM1", "DM23", "PM0", "PM1", "PM23","PN1","PN2", "PN03"],#"SM0","SPM0","SM1","SPM1","SM23","SPM23"],
    default="E2023")                   # the default value is a string

# Ignore the warning

In [43]:
elec = newSwitchAct(USER_DB, "elec", p_electricity, {
    "DM0" : Elec_FR_2050_DM0,
    "DM1" :Elec_FR_2050_DM1,
    "DM23" : Elec_FR_2050_DM23,
    "PM0": Elec_FR_2050_PM0,
    "PM1": Elec_FR_2050_PM1,
    "PM23": Elec_FR_2050_PM23,
    "PN1": Elec_FR_2050_PN1,
    "PN2": Elec_FR_2050_PN2,
    "PN03": Elec_FR_2050_PN03,
    "E2023":Elec_FR_2023,
    #"SDM0": Elec_solar_DM0,
    #"SPM0":Elec_solar_PM0,
    #"SDM1":Elec_solar_DM1,
    #"SPM1":Elec_solar_PM1,
    #"SDM23":Elec_solar_DM23,
    #"SPM23":Elec_solar_PM23,
    })

In [44]:
lifetime_stack_PEM = agb.newFloatParam(
    "lifetime_stack_PEM",
    distrib=agb.DistributionType.TRIANGLE,
    default=current_life_stack_PEM, min=current_life_stack_PEM, max=future_life_stack_PEM,
    label="lifetime of the PEM stack",
    description="evolution of the PEM lifetime stack",
    unit=" ")

In [45]:
lifetime_stack_AEC = agb.newFloatParam(
    "lifetime_stack_AEC",
    distrib=agb.DistributionType.TRIANGLE,
    default=current_life_stack_AEC, min=current_life_stack_AEC, max=future_life_stack_AEC,
    label="lifetime of the AEC stack",
    description="evolution of the PEM lifetime stack",
    unit=" ")

In [46]:
#Definition of the variability of the PEM efficiency
eff_PEM = agb.newFloatParam(
    "eff_PEM",
    distrib=agb.DistributionType.TRIANGLE,
    default=current_eff_PEM, min=current_eff_PEM, max=future_eff_PEM,
    label="efficiency of the PEM electrolyzer",
    description="evolution of the efficiency of the PEM electrolyser",
    unit=" ")

In [47]:
#Definition of the variability of the AEC efficiency
eff_AEC = agb.newFloatParam(
    "eff_AEC",
    distrib=agb.DistributionType.TRIANGLE,
    default=current_eff_AEC, min=current_eff_AEC, max=future_eff_AEC,
    label="efficiency of the AEC electrolyzer",
    description="evolution of the efficiency of the AEC electrolyser",
    unit=" ")

## Defining use phase

In [48]:
#hydrogen gas with an LHV of 120 MJ/kg, conversion factor 3.6MJ/kWh  >> efficiency = (1kg * 120MJ/kg)/(elec_use_phase * 3.6MJ/kWh)
#0.28kWh/0.75 = 0.373kWh > 0.28 is the heat listed in Gerloff's inventory, 0.75 is the efficiency to produce the heat by consuming electricity
elec_use_phase_PEM = 0.373 +(120/3.6)/eff_PEM 
#eff_PEM = 0.61 >> elec_use_phase =0.373+ 54.6 = 54.973  > Gerloff uses 54kW elec +0.28kWh heat
elec_use_phase_AEC = (120/3.6)/eff_AEC
#eff_AEC = 0.64 >> elec_use_phase =52.1  > Gerloff uses 51.8kW

In [49]:
def define_usephase_PEM():
    return agb.newActivity(USER_DB, "PEM usephase of H2, scale of 1MW system",
                          unit = "unit",
                          exchanges={
                              elec : elec_use_phase_PEM, 
                              water_H2 : water_demand_AEC 
                          })

usephase_PEM = define_usephase_PEM()
agb.printAct(usephase_PEM)

In [50]:
def define_usephase_AEC():
    return agb.newActivity(USER_DB, "AEC usephase of H2, scale of 1MW system",
                          unit = "unit",
                          exchanges={
                              elec : elec_use_phase_AEC, #no losses included
                              water_H2 : water_demand_AEC #no losses included
                          })

usephase_AEC = define_usephase_AEC()
agb.printAct(usephase_AEC)

## Defining electrolyzer and BoP production phase

In [51]:
#number of stacks is calculated with the plant's lifetime (20 years in hours = 175200h) and the stack lifetime
n_stacks_PEM=4
n_stacks_AEC=4

In [52]:
#divided by the total amount of H2 produced in 20y, as the technology lasts 20y
def define_production_PEM():
    return agb.newActivity(USER_DB, "PEM infra for production of H2, scale of 1MW system",
                          unit = "unit",
                          exchanges={
                              PEM_Stack:(n_stacks_PEM)/H2_produced_20_PEM,
                              PEM_BoP:1/H2_produced_20_PEM
                          })

production_PEM = define_production_PEM()
agb.printAct(production_PEM)

In [53]:

def define_production_AEC():
    return agb.newActivity(USER_DB, "AEC infra for production of H2, scale of 1MW system",
                          unit = "unit",
                          exchanges={
                              AEC_Stack:(n_stacks_AEC)/H2_produced_20_AEC,
                              AEC_BoP:1/H2_produced_20_AEC
                          })

production_AEC = define_production_AEC()
agb.printAct(production_AEC)

## Defining H2 global production system

In [54]:
def define_system_PEM():
    return agb.newActivity(USER_DB, name="PEM H2 system",
                            unit="unit",
                            exchanges={
                                usephase_PEM:1,
                                production_PEM:1
                            })

system_PEM=define_system_PEM()
agb.printAct(system_PEM)

In [55]:
#verify contribution of use and production phase:
agb.exploreImpacts(impacts[1],system_PEM)

In [56]:
def define_system_AEC():
    return agb.newActivity(USER_DB, name="AEC H2 system",
                            unit="unit",
                            exchanges={
                                usephase_AEC:1,
                                production_AEC:1
                            })

system_AEC=define_system_AEC()
agb.printAct(system_AEC)

## Calculating the impacts

In [57]:
result_table_PEM = agb.multiLCAAlgebric(
    system_PEM, 
    impacts,
    eff_PEM=current_eff_PEM,
    lifetime_stack_PEM=current_life_stack_PEM,
    p_electricity=["E2023", "DM0", "PM0", "DM1", "PM1", "DM23", "PM23", "PN1", "PN2", "PN03"]#"SM0","SPM0","SM1","SPM1","SM23","SPM23"]
)

df = pd.DataFrame(result_table_PEM)
# Save the DataFrame to an Excel file
df.to_excel("output_table_PEM_current.xlsx", index_label="CustomIndexLabel")

# Replace the default numbering with custom labels
result_table_PEM.index = ["E2023", "DM0", "PM0", "DM1", "PM1", "DM23", "PM23",  "PN1", "PN2", "PN03"]#"SM0","SPM0","SM1","SPM1","SM23","SPM23"]
result_table_PEM

In [58]:
result_table_PEM = agb.multiLCAAlgebric(
    system_PEM, 
    impacts,
    eff_PEM=future_eff_PEM,
    lifetime_stack_PEM=future_life_stack_PEM,
    p_electricity=["E2023", "DM0", "PM0", "DM1", "PM1", "DM23", "PM23",  "PN1", "PN2", "PN03"]#"SM0","SPM0","SM1","SPM1","SM23","SPM23"]
)

df = pd.DataFrame(result_table_PEM)
# Save the DataFrame to an Excel file
df.to_excel("output_table_PEM_future.xlsx", index_label="CustomIndexLabel")

# Replace the default numbering with custom labels
result_table_PEM.index = ["E2023", "DM0", "PM0", "DM1", "PM1", "DM23", "PM23", "PN1", "PN2", "PN03"]#"SM0","SPM0","SM1","SPM1","SM23","SPM23"]
result_table_PEM

In [59]:
result_table_AEC = agb.multiLCAAlgebric(
    system_AEC, 
    impacts,
    eff_AEC=current_eff_AEC,
    lifetime_stack_AEC=current_life_stack_AEC,
    p_electricity=["E2023", "DM0", "PM0", "DM1", "PM1", "DM23", "PM23",  "PN1", "PN2", "PN03"]
)

df = pd.DataFrame(result_table_AEC)
# Save the DataFrame to an Excel file
df.to_excel("output_table_AEC_current.xlsx", index_label="CustomIndexLabel")

# Replace the default numbering with custom labels
result_table_AEC.index = ["E2023", "DM0", "PM0", "DM1", "PM1", "DM23", "PM23",  "PN1", "PN2", "PN03"]
result_table_AEC

In [60]:
result_table_AEC = agb.multiLCAAlgebric(
    system_AEC, 
    impacts,
    eff_AEC=future_eff_AEC,
    lifetime_stack_AEC=future_life_stack_AEC,
    p_electricity=["E2023", "DM0", "PM0", "DM1", "PM1", "DM23", "PM23",  "PN1", "PN2", "PN03"]
)

df = pd.DataFrame(result_table_AEC)
# Save the DataFrame to an Excel file
df.to_excel("output_table_AEC_future.xlsx", index_label="CustomIndexLabel")

# Replace the default numbering with custom labels
result_table_AEC.index = ["E2023", "DM0", "PM0", "DM1", "PM1", "DM23", "PM23",  "PN1", "PN2", "PN03"]
result_table_AEC

In [61]:
agb.oat_matrix(system_PEM, impacts)

In [62]:
agb.oat_matrix(system_AEC, impacts)

In [63]:
agb.freezeParams(db_name=USER_DB)
bwa.print_recursive_calculation(AEC_Stack, lcia_method = climate)

In [64]:
H_PM0_PEM= agb.findActivity("hydrogen production, gaseous, 30 bar, from PEM electrolysis, from grid electricity, domestic, FE2050*", single= False, db_name=ECO_DB)

In [ ]:
result_h = agb.multiLCAAlgebric(
    H_PM0_PEM, 
    impacts,
    )
result_h